### Initialize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
from transformers import RobertaForSequenceClassification, RobertaTokenizer, Trainer, TrainingArguments
from datasets import load_metric
import pandas as pd
import torch.nn as nn

# Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-large-mnli')
model = RobertaForSequenceClassification.from_pretrained('roberta-large-mnli')
model.classifier.out_proj = nn.Linear(model.config.hidden_size, 2)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.num_labels = 2

In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

### Preprocess datasets


In [ ]:
# # Concat training data
# df_pap = pd.read_csv("/content/drive/MyDrive/datasets/augmented_processed/pap_train_processed_augmented.csv")
# df_pep = pd.read_csv("/content/drive/MyDrive/datasets/augmented_processed/pep_train_processed.csv")
# # Concat pap(unfiltered) with pep-3k vertically
# concatenated_df = pd.concat([df_pap, df_pep], ignore_index=True)
# # concatenated_df
# concatenated_df.to_csv('/content/drive/MyDrive/datasets/augmented_processed/concatenated_train.csv', index=False, sep=',', header=True, encoding='utf-8')


# # Concat dev data
# df_pap_val = pd.read_csv("/content/drive/MyDrive/datasets/augmented_processed/pap_dev_processed_augmented.csv")
# df_pep_val = pd.read_csv("/content/drive/MyDrive/datasets/augmented_processed/pep_dev_processed.csv")
# # Concat pap(unfiltered) with pep-3k vertically
# concatenated_df_val = pd.concat([df_pap_val, df_pep_val], ignore_index=True)
# # concatenated_df
# concatenated_df_val.to_csv('/content/drive/MyDrive/datasets/augmented_processed/concatenated_dev.csv', index=False, sep=',', header=True, encoding='utf-8')


# # Concat test data
# df_pap_test = pd.read_csv("/content/drive/MyDrive/datasets/augmented_processed/pap_test_processed_augmented.csv")
# df_pep_test = pd.read_csv("/content/drive/MyDrive/datasets/augmented_processed/pep_test_processed.csv")
# # Concat pap(unfiltered) with pep-3k vertically
# concatenated_df_test = pd.concat([df_pap_test, df_pep_test], ignore_index=True)
# # concatenated_df
# concatenated_df_test.to_csv('/content/drive/MyDrive/datasets/augmented_processed/concatenated_test.csv', index=False, sep=',', header=True, encoding='utf-8')

In [ ]:
concatenated_df = pd.read_csv('/content/drive/MyDrive/datasets/augmented_processed/concatenated_train.csv')
# Convert the DataFrame columns to a list
sentences = concatenated_df['text'].tolist()
labels = concatenated_df['label'].tolist()
# Tokenize sentences
#tokenized_sentences = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

concatenated_df_val = pd.read_csv('/content/drive/MyDrive/datasets/augmented_processed/concatenated_dev.csv')
# Convert the DataFrame columns to a list
sentences_val = concatenated_df_val['text'].tolist()
labels_val = concatenated_df_val['label'].tolist()
# Tokenize sentences
#tokenized_sentences_val = tokenizer(sentences_val, padding=True, truncation=True, return_tensors="pt")

concatenated_df_test = pd.read_csv('/content/drive/MyDrive/datasets/augmented_processed/concatenated_test.csv')
# Convert the DataFrame columns to a list
sentences_test = concatenated_df_test['text'].tolist()
labels_test = concatenated_df_test['label'].tolist()
# Tokenize sentences
#tokenized_sentences_test = tokenizer(sentences_test, padding=True, truncation=True, return_tensors="pt")

In [ ]:

# import matplotlib.pyplot as plt

# # Count the frequency of each label
# value_counts = concatenated_df['label'].value_counts()

# # Plotting the bar chart
# value_counts.plot(kind='bar')
# plt.title('Training Data Label Distribution')
# plt.xlabel('Labels')
# plt.ylabel('Frequency')
# plt.xticks(rotation=0)  # Rotate labels for better readability
# plt.show()
# print(value_counts)


In [ ]:
# # Count the frequency of each label
# value_counts_val = concatenated_df_val['label'].value_counts()

# # Plotting the bar chart
# value_counts_val.plot(kind='bar')
# plt.title('Validation Data Label Distribution')
# plt.xlabel('Labels')
# plt.ylabel('Frequency')
# plt.xticks(rotation=0)  # Rotate labels for better readability if necessary
# plt.show()
# print(value_counts_val)

In [ ]:
# # Count the frequency of each label
# value_counts_test = concatenated_df_test['label'].value_counts()

# # Plotting the bar chart
# value_counts_test.plot(kind='bar')
# plt.title('Test Data Label Distribution')
# plt.xlabel('Labels')
# plt.ylabel('Frequency')
# plt.xticks(rotation=0)  # Rotate labels for better readability if necessary
# plt.show()
# print(value_counts_test)

### DataLoader

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True)
        return { 'input_ids': encoding['input_ids'].squeeze(0),  # Squeeze to remove batch dimension
                 'attention_mask': encoding['attention_mask'].squeeze(0),
                 'labels': torch.tensor(label, dtype=torch.long) }  # Ensure labels are long tensors

# Create dataset and data loader
dataset = CustomDataset(sentences, labels, tokenizer)
dataset_val = CustomDataset(sentences_val, labels_val, tokenizer)
# Create DataLoaders
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=4, shuffle=False)


### Training process

In [ ]:
import torch
from transformers import AdamW
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import StepLR

In [ ]:
# Check if GPU is available
#print("GPU is", "available" if torch.cuda.is_available() else "NOT AVAILABLE")

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Move model to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Setup the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Learning Rate Scheduler (example: step decay)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

# Number of epochs
num_epochs = 20

# Initialize the variable to track the best validation loss
best_val_loss = float('inf')



# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0
    for i, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        #print("input_ids shape:", input_ids.shape)
        #print("labels shape:", labels.shape)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # Print loss every 10 (or N) batches
        #if i % 10 == 0:
         #   print(f"Epoch {epoch + 1}/{num_epochs}, Batch {i}, Current Loss: {loss.item()}")

    average_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} | Train Loss: {average_train_loss}")

    # Update learning rate
    scheduler.step()

    # Validation phase
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            val_loss += loss.item()

    average_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} | Validation Loss: {average_val_loss}")

    # After the end of each epoch, check if the validation loss improved
    if average_val_loss < best_val_loss:
        print(f"Validation Loss Improved ({best_val_loss} --> {average_val_loss})")
        best_val_loss = average_val_loss

        # Save the model state
        torch.save(model.state_dict(), '/content/drive/MyDrive/datasets/augmented_processed/best_model_state.pth')
        print("Model saved as best_model_state.pth")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20 | Train Loss: 0.7144558868258706
Epoch 1/20 | Validation Loss: 0.6939042361525746
Validation Loss Improved (inf --> 0.6939042361525746)
Model saved as best_model_state.pth
Epoch 2/20 | Train Loss: 0.7092854306587179
Epoch 2/20 | Validation Loss: 0.6942091735152455
Epoch 3/20 | Train Loss: 0.7063648732749181
Epoch 3/20 | Validation Loss: 0.6936819038607858
Validation Loss Improved (0.6939042361525746 --> 0.6936819038607858)
Model saved as best_model_state.pth
Epoch 4/20 | Train Loss: 0.7028192445781409
Epoch 4/20 | Validation Loss: 0.6932887647833142
Validation Loss Improved (0.6936819038607858 --> 0.6932887647833142)
Model saved as best_model_state.pth
Epoch 5/20 | Train Loss: 0.7060924486199497
Epoch 5/20 | Validation Loss: 0.6978877244831679
Epoch 6/20 | Train Loss: 0.7019965180259574
Epoch 6/20 | Validation Loss: 0.7380271832664291
Epoch 7/20 | Train Loss: 0.7020962899884494
Epoch 7/20 | Validation Loss: 0.6933415361813137
Epoch 8/20 | Train Loss: 0.7028426603696245
Epoch

### Evaluation

In [20]:
# Load the model
num_classes = 2
model = RobertaForSequenceClassification.from_pretrained('roberta-large', num_labels=num_classes)
checkpoint_path = '/content/drive/MyDrive/datasets/augmented_processed/best_model_state.pth'
model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cuda')))
model.eval()

predictions = []
probabilities = []  # for AUC-ROC

for text in concatenated_df_test['text']:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(model.device)
    with torch.no_grad():
        output = model(**inputs)
        logits = output.logits
        preds = torch.argmax(logits, dim=-1)
        probs = torch.softmax(logits, dim=-1)

        predictions.extend(preds.cpu().numpy())
        probabilities.extend(probs.cpu().numpy())



config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

accuracy = accuracy_score(concatenated_df_test['label'], predictions)
precision = precision_score(concatenated_df_test['label'], predictions, average='binary')
recall = recall_score(concatenated_df_test['label'], predictions, average='binary')
f1 = f1_score(concatenated_df_test['label'], predictions, average='binary')
auc_roc = roc_auc_score(concatenated_df_test['label'], [prob[1] for prob in probabilities])

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"AUC-ROC: {auc_roc}")


Accuracy: 0.5008130081300813
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC-ROC: 0.4838667033292441


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
